In [1]:
# yes it's a thing
import nltk
nltk.download("stopwords")

import argparse
import spacy
import gensim
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models import Phrases
from nltk.corpus import stopwords
import en_core_web_sm

[nltk_data] Downloading package stopwords to /Users/jlamb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
with open("/Users/jlamb/repos/w210/skills/sandbox/data_science/app_files/sample_resume.txt", "r") as f:
    user_input_text = f.read()

In [3]:
trigram_dict_file = "models/trigram_dictionary.dict"
bigram_model = "models/bigram_model_pos"
trigram_model = "models/trigram_model_pos"
lda_model = "models/lda_alpha_eta_auto_27"

In [4]:
trigram_dictionary = Dictionary.load(trigram_dict_file)

In [5]:
bigram_model = Phrases.load(bigram_model)

In [6]:
trigram_model = Phrases.load(trigram_model)

In [7]:
lda = LdaModel.load(lda_model)

In [8]:
nlp = spacy.load('en')

In [9]:
stopwords = stopwords.words('english')

topic_names = {1: u'Consulting and Contracting',
               2: u'DevOps',
               3: u'* Meta Job Description Topic: Students and Education',
               4: u'Finance and Risk',
               5: u'* Meta Job Description Topic: Benefits',
               6: u'* Meta Job Description Topic: Facebook Advertising',
               7: u'Aerospace and Flight Technology',
               8: u'* Meta Job Description Topic: Soft Skills',
               9: u'Product Management',
               10: u'Compliance and Process/Program Management',
               11: u'Project and Program Management',
               12: u'* Meta Job Description Topic: Generic',
               13: u'* Meta Job Description Topic: EO and Disability',
               14: u'Healthcare',
               15: u'Software Engineering and QA',
               16: u'Accounting and Finance',
               17: u'Human Resources and People',
               18: u'Sales',
               19: u'* Meta Job Description Topic: Startup-Focused',
               20: u'Federal Government and Defense Contracting',
               21: u'Web Development and Front-End Software Engineering',
               22: u'UX and Design',
               23: u'* Meta Job Description Topic: Education-Focused',
               24: u'Academic and Medical Research',
               25: u'Data Science',
               26: u'* Meta Job Description Topic: Non-Discrimination',
               27: u'Business Strategy'}

In [10]:
all_hard_skills = []
with open('models/hard_skills.txt', 'r') as infile:
    for line in infile:
        line = line.strip()
        all_hard_skills.append(line)

In [12]:
def vectorize_input(input_doc, bigram_model, trigram_model, trigram_dictionary):
    """
    (1) parse input doc with spaCy
    (2) apply text pre-proccessing steps,
    (3) create a bag-of-words representation
    (4) create an LDA representation
    """

    # parse the review text with spaCy
    parsed_doc = nlp(input_doc)

    # lemmatize the text and remove punctuation and whitespace
    unigram_doc = []
    for token in parsed_doc:
        if not (token.is_punct or token.is_space):
            unigram_doc.append(token.lemma)

    # apply the first-order and secord-order phrase models
    bigram_doc = bigram_model[unigram_doc]
    trigram_doc = trigram_model[bigram_doc]

    # remove any remaining stopwords
    trigram_review = [term for term in trigram_doc
                      if not term in stopwords]

    # create a bag-of-words representation
    doc_bow = trigram_dictionary.doc2bow(trigram_doc)

    # create an LDA representation
    document_lda = lda[doc_bow]
    return trigram_review, document_lda

In [13]:
def top_match_list(document_lda, topic_names, num_terms=100):
    '''
    Take the above results and just save to a list of the top 500 terms in the topic.
    Also return the user's highest probability topic, along with the probability itself.
    '''
    sorted_doc_lda = sorted(document_lda, key=lambda review_lda: -review_lda[1])
    topic_number = sorted_doc_lda[0][0]
    freq = sorted_doc_lda[0][1]
    highest_probability_topic = topic_names[topic_number + 1]
    top_topic_skills = []
    for term, term_freq in lda.show_topic(topic_number, topn=num_terms):
        top_topic_skills.append(term)
    return top_topic_skills, highest_probability_topic, round(freq, 3)

In [14]:
def common_skills(top_topic_skills, user_skills):
    return [item for item in top_topic_skills if item in user_skills]

In [15]:
def non_common_skills(top_topic_skills, user_skills):
    return [item for item in top_topic_skills if item not in user_skills]

In [16]:
def get_skills(text_document, num_skills):
    '''
    Output the following objects in a Python dictionary:
    1. top_topic: Name of user's highest-percentage-match topics (str)
    2. match_percent: The percentage match (number between 0 and 1) (float)
    3. skills_in_common: List of ALL skills user has in common with topic
    4. skills_not_in_common: List of ALL skills user does NOT have in common with topic
    5. hard_skills_in_common: List of HARD skills user has in common with topic
    6. hard_skills_not_in_common: List of HARD skills user does NOT have in common with topic
    '''
    user_skills, my_lda = vectorize_input(text_document,
                                          bigram_model,
                                          trigram_model,
                                          trigram_dictionary)
    skills_list, top_topic, percent_match = top_match_list(my_lda,
                                                           topic_names,
                                                           num_terms=500)

    hard_skills_list = [skill for skill in skills_list if skill in all_hard_skills]
    output_dict = {
      "top_topic": top_topic,
      "percent_match": percent_match,
      "skills": {
          "all": {
              "has": common_skills(skills_list, user_skills)[:num_skills],
              "missing": non_common_skills(skills_list, user_skills)[:num_skills]
          },
          "hard": {
              "has": common_skills(hard_skills_list, user_skills)[:num_skills],
              "missing": non_common_skills(hard_skills_list, user_skills)[:num_skills]
          }
      }
    }
    return output_dict

In [20]:
user_skills, my_lda = vectorize_input(user_input_text,
                                          bigram_model,
                                          trigram_model,
                                          trigram_dictionary)

/Users/jlamb/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


TypeError: 'int' object is not iterable

In [21]:
bigram_model

In [22]:
parsed_doc = nlp(user_input_text)

In [24]:
unigram_doc = []
for token in parsed_doc:
    if not (token.is_punct or token.is_space):
        unigram_doc.append(token.lemma)

In [26]:
bigram_doc = bigram_model[unigram_doc]

/Users/jlamb/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [28]:
bigram_doc

In [36]:
bigram_doc

In [37]:
trigram_doc = trigram_model[bigram_doc]

/Users/jlamb/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


TypeError: 'int' object is not iterable